#**Aivle 스쿨 지원 질문, 답변 챗봇 만들기**
# 단계2 : 모델링

## 0.미션

* 다음 세가지 챗봇을 만들고 비교해 봅시다.
* 챗봇1. Word2Vec 임베딩 벡터 기반 머신러닝 분류 모델링
    * Word2Vec 모델을 만들고 임베딩 벡터를 생성합니다.
    * 임베딩 벡터를 이용하여 intent를 분류하는 모델링을 수행합니다.
        * 이때, LightGBM을 추천하지만, 다른 알고리즘을 이용할수 있습니다.
    * 예측된 intent의 답변 중 임의의 하나를 선정하여 출력합니다.
* 챗봇2. 단계별 모델링1
    * 1단계 : type(일상대화 0, 에이블스쿨Q&A 1) 분류 모델 만들기
        * Embedding + LSTM 모델링
    * 2단계 : 사전학습된 Word2Vec 모델을 로딩하여 train의 임베딩벡터 저장
    * 코사인 유사도로 intent 찾아 답변 출력
        * 새로운 문장의 임베딩벡터와 train의 임베딩 벡터간의 코사인 유사도 계산
        * 가장 유사도가 높은 질문의 intent를 찾아 답변 출력하기
* 챗봇3. 단계별 모델링2
    * 1단계 : 챗봇2의 1단계 모델을 그대로 활용
    * 2단계 : FastText 모델 생성하여 train의 임베딩벡터 저장
    * 코사인 유사도로 intent 찾아 답변 출력
        * 새로운 문장의 임베딩벡터와 train의 임베딩 벡터간의 코사인 유사도 계산
        * 가장 유사도가 높은 질문의 intent를 찾아 답변 출력하기

* 챗봇3개에 대해서 몇가지 질문을 입력하고 각각의 답변을 비교해 봅시다.


## 1.환경준비

### (1)라이브러리 설치

#### 1) gensim 3.8.3 설치

In [1]:
#gensim은 자연어 처리를 위한 오픈소스 라이브러리입니다. 토픽 모델링, 워드 임베딩 등 다양한 자연어 처리 기능을 제공
# 현재 4.x 버전이 최신이지만, 3.8.3 버전으로 진행
!pip install gensim==3.8.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


* [코랩] 위 라이브러리 설치후 런타임 재시작 필요!

#### 2) 형태소 분석을 위한 라이브러리

In [2]:
# mecab 설치를 위한 관련 패키지 설치
!apt-get install curl git
!apt-get install build-essential
!apt-get install cmake
!apt-get install g++
!apt-get install flex
!apt-get install bison
!apt-get install python-dev
!pip install cython
!pip install mecab-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68.0-1ubuntu2.18).
git is already the newest version (1:2.25.1-1ubuntu3.10).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.16.3-1ubuntu1.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:9.3.0-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree       
Rea

In [3]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu: linux, mac os 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


### (2) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 필요하다고 판단되는 라이브러리를 추가하세요.
import os

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import * 

import tensorflow as tf
from keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from keras import Input, Model
from keras import optimizers
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics.pairwise import cosine_similarity

import random

* 형태소 분석을 위한 함수를 제공합니다.

In [5]:
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

# 다양한 토크나이저를 사용할 수 있는 함수
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        # "kkma":
        tokenizer = Kkma()
        
    return tokenizer

In [6]:
# 형태소 분석을 수행하는 함수

def tokenize(tokenizer_name, original_sent, nouns=False):
    # 미리 정의된 몇 가지 tokenizer 중 하나를 선택
    tokenizer = get_tokenizer(tokenizer_name)

    # tokenizer를 이용하여 original_sent를 토큰화하여 tokenized_sent에 저장하고, 이를 반환합니다.
    sentence = original_sent.replace('\n', '').strip()
    if nouns:       
        # tokenizer.nouns(sentence) -> 명사만 추출
        tokens = tokenizer.nouns(sentence)
    else:
        tokens = tokenizer.morphs(sentence)
    tokenized_sent = ' '.join(tokens)
    
    return tokenized_sent

### (3) 데이터 로딩
* 전처리 단계에서 생성한 데이터들을 로딩합니다.
    * train, test
    * 형태소분석 결과 데이터 : clean_train_questions, clean_test_questions

* 구글 드라이브 연결

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
path = '/content/drive/MyDrive/AIVLE/4월/mini6/'

* 저장된 .pkl 파일들을 불러옵니다.
* 불러 온 후에는 shape를 확인해 봅시다.

In [9]:
train = joblib.load(path+'train.joblib')
test = joblib.load(path+'test.joblib')

clean_train_questions = joblib.load(path+'clean_train_questions.joblib')
clean_test_questions = joblib.load(path+'clean_test_questions.joblib')

In [10]:
train.head()

,intent,Q,A,type,Q_len
0,0,떨어뜨려서 핸드폰 액정 나갔어,as 맡기세요.,0,16
1,0,액정 나갔어,as 맡기세요.,0,6
2,0,핸드폰 떨어뜨려서 고장 났나봐,as 맡기세요.,0,16
3,1,노트북이 작동이 안되네,AS센터에 맡겨보세요.,0,12
4,1,노트북 키보드가 안먹히네,AS센터에 맡겨보세요.,0,13


In [11]:
clean_train_questions

0                                 [떨어뜨려서, 핸드폰, 액정, 나갔, 어]
1                                             [액정, 나갔, 어]
2                               [핸드폰, 떨어뜨려서, 고장, 났, 나, 봐]
3                                [노트북, 이, 작동, 이, 안, 되, 네]
4                                 [노트북, 키보드, 가, 안, 먹히, 네]
                              ...                        
1293    [KT, 에, 이, 블, 스쿨, 만, 의, 장점, 이, 무엇, 인지, 상세히, 설명...
1294                   [다른, 교육, 과, 의, 차이점, 은, 무엇, 인가요, ?]
1295                        [다른, 교육, 과, 차이점, 이, 있, 나요, ?]
1296     [다른, 국비, 지원, 교육, 보다, 특별, 한, 점, 이, 있, 는지, 궁금, 해요]
1297                  [다른, 교육, 보다, 특별, 한, 것, 이, 있, 나요, ?]
Name: Q, Length: 1296, dtype: object

## 2.챗봇1

* **상세요구사항**
    * Word2Vec을 활용한 LightGBM 모델링(intent 분류)
        * Word2Vec을 이용하여 임베딩벡터 생성하기
            * Word Embedding으로 문장벡터 구하기
        * 임베딩 벡터를 이용하여 ML기반 모델링 수행하기
            * LightGBM 권장(다른 알고리즘을 이용할수 있습니다.)
    * 챗봇 : 모델의 예측결과(intent)에 따라 답변하는 챗봇 만들기
        * 질문을 입력받아, 답변하는 함수 생성

### (1) Word2Vec을 이용하여 임베딩벡터 생성하기
* 'mecab' 형태소 분석기를 이용하여 문장을 tokenize
    * Word2Vec 모델을 만들기 위해서 입력 데이터는 리스트 형태여야 합니다.
    * 그래서 다시 리스트로 저장되도록 토크나이즈 해 봅시다.
* Word Embedding으로 문장벡터를 생성합니다.
    * 먼저 Word2Vec 모델을 만들고, train의 질문들을 문장벡터로 만듭시다.


#### 1) 'mecab' 형태소 분석기를 이용하여 문장을 tokenize

In [12]:
train_mecab = list(train['Q'])
print(train_mecab)

['떨어뜨려서 핸드폰 액정 나갔어', '액정 나갔어', '핸드폰 떨어뜨려서 고장 났나봐', '노트북이 작동이 안되네', '노트북 키보드가 안먹히네', '노트북 전원이 안들어와.', '노트북을 떨어뜨려서 고장 난 것 같아.', '컴퓨터가 제대로 작동하지 않아요', '컴퓨터가 에러 메시지를 띄우고 있어요.', '컴터 고장났나봐', '컴터가 맛이 갔어', '컴터가 안돼', '핸드폰 고장났나봐', '핸드폰 맛이 갔어', '핸드폰 안돼', '노트북 안돼', '내 의지는 상관없나봐', '내 의지로 안되는 일인가봐', '건강이 최고', '건강이 최고인 것 같아', '아프면 안 되는데', '올해도 건강하길', '올해도 행복하길', '올해 취업하길', '올해 합격하길', '건강이 최고야', '올해는 꼭 취업할거야', '이번년도는 꼭 집을 장만할거야', '여자친구를 사귀고 싶어', '올해도 건강하고 행복하게 살거야', '내가 잘하는 일을 찾고 싶어', '내가 좋아하는 일은 무엇인지 찾아볼려고', '비 맞아서 옷 젖었어', '옷이 다 젖었어', '열나고 아파', '코맹맹', '동상 걸릴 뻔했어', '동상 걸릴거 같아.', '바람 많이 부네', '바람이 너무 많이 부네', '생각보다 오늘 너무 춥네', '아놔 코 막혀', '얼어 죽는 줄', '얼어죽는 줄 알았어', '오늘 생각보다 춥네', '추워 죽을 뻔했네', '추워죽겠다', '코 막혀', '비 맞아서 옷 젖었어', '옷이 다 젖었어', '열나고 아파', '코맹맹', '동상 걸릴 뻔했어', '동상 걸릴거 같아.', '바람 많이 부네', '바람이 너무 많이 부네', '생각보다 오늘 너무 춥네', '아놔 코 막혀', '얼어 죽는 줄', '얼어죽는 줄 알았어', '오늘 생각보다 춥네', '추워 죽을 뻔했네', '추워죽겠다', '코 막혀', '오늘 날씨가 생각보다 추워', '코가 막혀서 싫어', '너무 추워서 몸이 얼어붙을 것 같아', '너무 추워서 죽을 것 같았어', '오늘 날씨가 생각보다 차갑네', '추워서 죽을 것 같아', '코가

#### 2) Word Embedding으로 문장벡터 구하기
* Word2Vec
    * 위에서 저장한 입력 데이터를 사용하여 Word2Vec 모델이 생성
    * 모델은 size(단어 벡터의 차원), 
    * window(컨텍스트 창의 크기), 
    * max_vocab_size(고려할 최대 어휘 크기), 
    * min_count(포함할 단어의 최소 빈도)와 같은 특정 하이퍼파라미터로 훈련됩니다.
    * sg : 사용할 훈련 알고리즘 - 1은 skip-gram, 0은 CBOW )

In [13]:
train['Q'][0]

'떨어뜨려서 핸드폰 액정 나갔어'

In [14]:
from gensim.models import Word2Vec

# Word2Vec 모델 생성
wv_model = Word2Vec(sentences=train_mecab, size=100, window=5, min_count=5, sg=-1)

In [15]:
%%time
# train의 질문들을 리스트로 변환
raw_sentences = list(train['Q'])

# train_sentences 리스트 생성 
train_sentences = []

# raw_sentences에서 하나씩 가져와 토큰화하며 추가하기
for sent in raw_sentences:
    # 토크나이즈 함수를 통해 'mecab' 형태소 분석기를 이용해 문장 토큰화
    # split 함수를 이용해 토큰화된 단어들을 리스트로 만들어 추가
    train_sentences.append(tokenize('mecab', sent).split(' '))

# train_sentences에서 처음 다섯개의 리스트를 출력
print(train_sentences[:5])

[['떨어뜨려서', '핸드폰', '액정', '나갔', '어'], ['액정', '나갔', '어'], ['핸드폰', '떨어뜨려서', '고장', '났', '나', '봐'], ['노트북', '이', '작동', '이', '안', '되', '네'], ['노트북', '키보드', '가', '안', '먹히', '네']]
CPU times: user 1.37 s, sys: 947 ms, total: 2.31 s
Wall time: 4.38 s


In [16]:
# word2vec으로 임베딩할때, 워드 벡터의 차원 수를 지정하는 변수로, 100이 기본값
SIZE = 100
# word2vec 모델에서 한 단어의 좌우에 있는 최대 단어 개수를 지정하는 변수로 3으로 지정되어 있음
WINDOW = 3
# word2vec 모델에 의해 무시될 최소 단어 빈도 수, 해당 값보다 적게 나타나는 단어는 무시됨됨
MIN_COUNT = 3

In [17]:
from gensim.models import Word2Vec

#Word2Vec 모델 생성
wv_model = Word2Vec(train_sentences, # 학습할 문장
                    size = SIZE, # 워드 임베딩 벡터의 크기
                    window = WINDOW, # 문맥 윈도우 크기
                    max_vocab_size = 500, # 학습에 사용할 최대 단어 수 
                    min_count = MIN_COUNT, # 학습에 사용할 최소 단어 수
                    workers = 4, # 학습을 위한 프로세스 수
                    iter = 10, # 학습 횟수
                    sg = 1) # Skip-Gram 방식을 사용
                    

* Word2Vec 모델로부터 데이터를 벡터화하기 위한 함수 생성

In [18]:
# Word2Vec 모델로부터 하나의 문장을 벡터화 시키는 함수
def get_sent_embedding(model, embedding_size, tokenized_words):
    # 임베딩 벡터를 0으로 초기화
    feature_vec = np.zeros((embedding_size,), dtype='float32')
    # 단어 개수 초기화
    n_words = 0
    # 모델 단어 집합 생성
    index2word_set = set(model.wv.index2word)
    # 문장의 단어들을 하나씩 반복
    for word in tokenized_words:
        # 모델 단어 집합에 해당하는 단어일 경우에만
        if word in index2word_set:
            # 단어 개수 1 증가
            n_words += 1
            # 임베딩 벡터에 해당 단어의 벡터를 더함
            feature_vec = np.add(feature_vec, model[word])
    # 단어 개수가 0보다 큰 경우 벡터를 단어 개수로 나눠줌 (평균 임베딩 벡터 계산)
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [19]:
# 문장벡터 데이터 셋 만들기
def get_dataset(sentences, model, num_features):
    dataset = list()

    # 각 문장을 벡터화해서 리스트에 저장
    for sent in sentences:
        dataset.append(get_sent_embedding(model, num_features, sent))

    # 리스트를 numpy 배열로 변환하여 반환
    sent_embedding_vectors = np.stack(dataset)
    
    return sent_embedding_vectors

* 이제 학습데이터의 Q를 Word2Vec 모델을 사용하여 벡터화 합니다.

In [20]:
# 학습 데이터의 문장들을 Word2Vec 모델을 사용하여 벡터화
# train_data_vecs = get_dataset(             )

train_data_vecs = get_dataset(train_sentences, wv_model, SIZE)
train_data_vecs.shape

(1296, 100)

* 훈련된 Word2Vec 모델을 사용하여 문장 목록에 대한 문장 임베딩을 생성하고 이를 2차원 numpy 배열에 저장합니다. 
* 그런 다음 이러한 임베딩을 다양한 기계 학습 모델의 입력 기능으로 사용할 수 있습니다

#### 3) 다른 방법으로 토큰화 하기

In [21]:
train_mecab[0]

'떨어뜨려서 핸드폰 액정 나갔어'

In [22]:
import nltk
nltk.download('stopwords')
tk_morphs = nltk.Text('morphs')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
tk_morphs.fit_on_texts(train_mecab)

AttributeError: ignored

### (2) 분류 모델링
* 데이터 분할
    * x, y
        * x : 이전 단계에서 저장된 임베딩벡터(train_data_vecs)
        * y : intent 값들
    * train, val
        * train_test_split 활용
* 머신러닝 모델링
    * lightGBM, RandomForest 등을 활용하여 학습
    * 필요하다면 hyper parameter 튜닝을 시도해도 좋습니다.
* validation set으로 검증해 봅시다.

In [24]:
x = train_data_vecs
y = np.array(list(train['intent']))

# train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
display(np.unique(y_val, return_counts=True))
display(np.unique(y_val, return_counts=True)[1].shape)

# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)
# display(np.unique(y_val, return_counts=True))
# display(np.unique(y_val, return_counts=True)[1].shape)

(array([  1,   3,   4,   5,   6,   9,  10,  12,  20,  22,  23,  28,  29,
         31,  33,  35,  36,  38,  39,  40,  41,  43,  44,  46,  47,  50,
         52,  53,  54,  55,  57,  58,  59,  61,  64,  65,  69,  70,  71,
         72,  73,  75,  81,  82,  84,  86,  87,  88,  90,  91,  93,  94,
         95,  96,  97,  99, 100, 102, 103, 104, 105, 106, 107, 108, 109,
        110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
        123, 124, 125, 126, 128, 136, 138]),
 array([ 1,  3,  3,  1,  8,  2,  1,  1,  3,  3,  1,  1,  1,  1,  1,  2,  1,
         1,  3,  2,  2,  1,  2,  1,  1,  2,  2,  1,  1,  2,  1,  2,  1,  1,
         2,  3,  3,  3,  1,  2,  1,  3,  1,  1,  1,  1,  1,  4,  2,  1,  4,
         1,  1,  1,  1,  1,  3,  2,  5,  1, 29,  2,  4,  5,  5,  3,  4, 16,
         3,  3,  6,  3,  9,  2, 10, 15,  3,  4,  5,  7,  2,  3,  5,  1,  1]))

(85,)

* 모델1

In [25]:
%%time
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)
rf_pred = model.predict(x_val)
precision_score(y_val, rf_pred, average='macro')

CPU times: user 4 s, sys: 157 ms, total: 4.15 s
Wall time: 8.18 s


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.3146887400793651

* 모델2

In [26]:
%%time
from lightgbm import LGBMClassifier

model = LGBMClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_val)
precision_score(y_val, y_pred, average='macro')

CPU times: user 2min 1s, sys: 911 ms, total: 2min 2s
Wall time: 1min 22s


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.2906799827852459

### (3) 챗봇 구축

* **상세요구사항**
    * 챗봇 flow : input 질문 -> 분류 모델로 intent 예측 --> intent에 해당하는 답변 출력
        * 하나의 intent 에는 여러 답변이 있습니다. 이중 한가지를 랜덤하게 선택합니다.

#### 1) 데이터 중 하나에 대해서 테스트

In [27]:
    a = input('put your text here:') # random.randrange(0, len(train))
    # pred = input('put your model here: (rf, lgbm)')
    # if pred == 'lgbm':


    a = [tokenize(get_tokenizer('mecab'), a)]

    # 토큰화된 데이터를 입력으로 사용하여 Word2Vec 모델 학습
    wv_model = Word2Vec(a, size=100, window=5, min_count=1, workers=4, sg=1)

    # 학습된 Word2Vec 모델을 사용하여 데이터셋 구성
    inputQuestion = get_dataset(a, wv_model, 100)

    import random

    predicted_intent = model.predict(inputQuestion)

    # 예측된 intent에 해당하는 답변들의 리스트 가져오기
    response_options = train[train['intent'] == predicted_intent[0]]['A'].tolist()

    # 답변들 중에서 랜덤하게 하나 선택
    response = random.choice(response_options)

    print("답변:", response)

put your text here:어떤 특징이 있나요?


답변: 맛있게 드세요.


#### 2) 챗봇 함수 만들기
* 테스트 코드를 바탕으로 질문을 받아 답변을 하는 함수를 생성합시다.
* 성능이 좋은 모델 사용. 

In [ ]:
def get_answer1(question): 




    return 

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

In [ ]:
sns.heatmap(confusion_matrix(y_val, y_pred), cmap='Blues')

## 3.챗봇2

* **세부요구사항**
    * 단계별 챗봇을 만들어 봅시다.
        * 1단계 : type을 0과 1로 분류하는 모델 생성(Embedding + LSTM 모델)
        * 2단계 : 
            * 각 type에 맞게, 사전학습된 Word2Vec 모델을 사용하여 임베딩 벡터(train)를 만들고
        * 3단계 : 챗봇 만들기
            * input 문장과 train 임베딩 벡터와 코사인 유사도 계산
            * 가장 유사도가 높은 질문의 intent 찾아
            * 해당 intent의 답변 중 무작위로 하나를 선정하여 답변하기

### (1) 1단계 : type 분류 모델링(LSTM)
- LSTM

#### 1) 데이터 준비
* 학습용 데이터를 만들어 봅시다.
    * 시작 데이터 : clean_train_questions, clean_test_questions
    * 각 토큰에 인덱스를 부여하는 토크나이저를 만들고 적용
        * from tensorflow.keras.preprocessing.text import Tokenizer 를 사용
    * 문장별 길이에 대한 분포를 확인하고 적절하게 정의.

In [124]:
clean_train_questions.head()

0      [떨어뜨려서, 핸드폰, 액정, 나갔, 어]
1                  [액정, 나갔, 어]
2    [핸드폰, 떨어뜨려서, 고장, 났, 나, 봐]
3     [노트북, 이, 작동, 이, 안, 되, 네]
4      [노트북, 키보드, 가, 안, 먹히, 네]
Name: Q, dtype: object

In [125]:
clean_test_questions.head()

14                             [AS, 센터, 에, 맡겨, 보, 세요, .]
678    [KT, 에, 이, 블, 스쿨, 은, 미취, 업자, 를, 대상, 으로, 하, 며, ...
Name: A, dtype: object

In [126]:
# 각각의 토큰에 인덱스 부여하는 토크나이저 선언
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()

# .fit_on_tests 이용하여 토크나이저 만들기
tokenizer.fit_on_texts(clean_train_questions)
word_dic = tokenizer.word_index
print(word_dic)

word_seq = tokenizer.texts_to_sequences(clean_train_questions)
print(word_seq)

max_len = 0
for i in word_seq:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

word_seq_padded = pad_sequences(word_seq, maxlen=len_seq)
print(word_seq_padded)

{'?': 1, '나요': 2, '이': 3, '있': 4, '하': 5, '는': 6, '가': 7, '교육': 8, '지원': 9, '을': 10, '에': 11, '수': 12, '되': 13, '어': 14, '할': 15, '.': 16, '도': 17, '경우': 18, '한가요': 19, '은': 20, '고': 21, '가능': 22, '어떻게': 23, '를': 24, '면': 25, ',': 26, '으로': 27, '없': 28, '나': 29, '중': 30, '로': 31, '해야': 32, '궁금': 33, '서류': 34, '는데': 35, '한': 36, '합니다': 37, '어떤': 38, '에서': 39, '노트북': 40, '졸업': 41, '다': 42, '싶': 43, '과': 44, '의': 45, '해': 46, '진행': 47, '인': 48, '과정': 49, '인가요': 50, '시': 51, '먹': 52, '수강': 53, '만': 54, '받': 55, '채용': 56, '안': 57, '다른': 58, '아': 59, '지': 60, '주': 61, '대면': 62, '연계': 63, '게': 64, '보': 65, '했': 66, '좋': 67, '자': 68, '공부': 69, '제공': 70, '지역': 71, '제출': 72, '합격': 73, '거': 74, '시간': 75, '재': 76, '증명서': 77, '것': 78, '들': 79, '기준': 80, '추가': 81, '검사': 82, '너무': 83, '뭔가요': 84, '수료': 85, '같': 86, '일': 87, '취업': 88, '와': 89, '미취': 90, '업자': 91, '테스트': 92, '뭐': 93, '참여': 94, '을까요': 95, '비대': 96, '생': 97, 'kt': 98, '장소': 99, '겠': 100, '기간': 101, '야': 102, '오늘': 103, '해도': 104, '아르바이트':

In [127]:
print(word_dic)

{'?': 1, '나요': 2, '이': 3, '있': 4, '하': 5, '는': 6, '가': 7, '교육': 8, '지원': 9, '을': 10, '에': 11, '수': 12, '되': 13, '어': 14, '할': 15, '.': 16, '도': 17, '경우': 18, '한가요': 19, '은': 20, '고': 21, '가능': 22, '어떻게': 23, '를': 24, '면': 25, ',': 26, '으로': 27, '없': 28, '나': 29, '중': 30, '로': 31, '해야': 32, '궁금': 33, '서류': 34, '는데': 35, '한': 36, '합니다': 37, '어떤': 38, '에서': 39, '노트북': 40, '졸업': 41, '다': 42, '싶': 43, '과': 44, '의': 45, '해': 46, '진행': 47, '인': 48, '과정': 49, '인가요': 50, '시': 51, '먹': 52, '수강': 53, '만': 54, '받': 55, '채용': 56, '안': 57, '다른': 58, '아': 59, '지': 60, '주': 61, '대면': 62, '연계': 63, '게': 64, '보': 65, '했': 66, '좋': 67, '자': 68, '공부': 69, '제공': 70, '지역': 71, '제출': 72, '합격': 73, '거': 74, '시간': 75, '재': 76, '증명서': 77, '것': 78, '들': 79, '기준': 80, '추가': 81, '검사': 82, '너무': 83, '뭔가요': 84, '수료': 85, '같': 86, '일': 87, '취업': 88, '와': 89, '미취': 90, '업자': 91, '테스트': 92, '뭐': 93, '참여': 94, '을까요': 95, '비대': 96, '생': 97, 'kt': 98, '장소': 99, '겠': 100, '기간': 101, '야': 102, '오늘': 103, '해도': 104, '아르바이트':

In [128]:
# 전체 토큰의 수로 vocab_size 지정
vocab_size = 1226

# fit_on_texts을 위에서 한번만 해도 되지만, vocab 사이즈를 확인하고 줄이거나 하는 시도를 할 수도 있기에 다시 수행


# .texts_to_sequences : 토크나이즈 된 데이터를 가지고 모두 시퀀스로 변환
x = word_seq_padded
y = train['type']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state=2023)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((1036, 57), (260, 57), (1036,), (260,))

In [129]:
# 각 토큰과 인덱스로 구성된 딕셔너리 생성,


# <PAD> 는 0으로 추가



* 문장별 토큰수에 대해 탐색적 분석을 수행해 봅시다.

* 문장별 토큰이 가장 큰 것이 57개 입니다. 

* 학습 입력을 위한 데이터 크기 맞추기
    * 문장이 짧기 때문에 MAX_SEQUENCE_LENGTH는 정하지 않아도 되지만,
    * 그러나 분포를 보고 적절하게 자릅시다.
    * 그리고 pad_sequences 함수를 이용하여 시퀀스데이터로 변환하기
* y는 train['type'] 와 test['type'] 입니다.

#### 2) 모델링

* 토크나이징 한 데이터를 입력으로 받아 
* Embedding 레이어와 LSTM 레이어를 결합하여 이진 분류 모델링을 수행합니다.

In [130]:
from tensorflow import keras

keras.backend.clear_session()

max_features = 2000   # 사용할 단어의 수
maxlen = 57   # 문장 최대 길이

# 모델 정의
binary_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features, 128, input_length=maxlen),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 모델 컴파일
binary_model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

binary_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 57, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 387,713
Trainable params: 387,713
Non-trainable params: 0
_________________________________________________________________


In [131]:
binary_model.fit(x_train, y_train,
          epochs = 10,
          batch_size = 32,
          validation_split=0.2)

Epoch 1/10
26/26 [==============================] - 10s 236ms/step - loss: 0.4464 - accuracy: 0.8321 - val_loss: 0.2072 - val_accuracy: 0.9471
Epoch 2/10
26/26 [==============================] - 3s 123ms/step - loss: 0.1176 - accuracy: 0.9758 - val_loss: 0.0489 - val_accuracy: 0.9952
Epoch 3/10
26/26 [==============================] - 3s 119ms/step - loss: 0.0350 - accuracy: 0.9940 - val_loss: 0.0406 - val_accuracy: 0.9904
Epoch 4/10
26/26 [==============================] - 3s 123ms/step - loss: 0.0177 - accuracy: 0.9976 - val_loss: 0.0099 - val_accuracy: 1.0000
Epoch 5/10
26/26 [==============================] - 5s 215ms/step - loss: 0.0071 - accuracy: 0.9988 - val_loss: 0.0315 - val_accuracy: 0.9952
Epoch 6/10
26/26 [==============================] - 3s 127ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 7/10
26/26 [==============================] - 3s 123ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch

In [132]:
score = binary_model.evaluate(x_val, y_val, batch_size=32)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

9/9 [==============================] - 1s 87ms/step - loss: 0.0029 - accuracy: 1.0000
Test loss: 0.002949046902358532
Test accuracy: 1.0


### (2) 사전학습모델(Word2Vec)

* Pre-trained Word2Vec model
    * 이미 다운로드 받아서 제공되었습니다.
        * ko.bin, ko.tsv
    * 참고 사이트: https://github.com/Kyubyong/wordvectors
        * 모델 파일 다운로드 사이트: https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view?resourcekey=0-Dq9yyzwZxAqT3J02qvnFwg
* 사전학습 모델을 로딩하고, 
* train 데이터셋의 질문(Q)을 임베딩벡터로 만들어, 열(Column)로 추가합니다.

#### 1) 모델 로딩
* 사전 학습된 모델을 로딩 : gensim.models.Word2Vec.load()
* 로딩 후 벡터 크기를 조회합시다. .vector_size

In [81]:
len(train), len(clean_train_questions)

(1296, 1296)

In [67]:
import gensim
pre_wv_model = gensim.models.Word2Vec.load(path + 'ko.bin')

In [70]:
# 모델의 벡터크기 조회
pre_wv_model.vector_size

200

#### 2) train 에 임베딩벡터 결과 저장
* get_sent_embedding 함수를 이용하여 train의 질문별 임베딩 결과를 저장합니다.
    * .apply(lambda .....) 를 활용하세요.

In [134]:
clean_train_vecs = get_dataset(clean_train_questions, pre_wv_model, pre_wv_model.vector_size)
clean_train_vecs.shape

(1296, 200)

In [135]:
ㅗ# x, y 데이터 생성
x = clean_train_vecs
y = np.array(list(train['intent']))

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=2023)
display(np.unique(y_val, return_counts=True))
display(np.unique(y_val, return_counts=True)[1].shape)

(array([  1,   3,   4,   5,   6,   9,  12,  13,  16,  19,  20,  21,  22,
         23,  26,  27,  28,  29,  30,  32,  35,  36,  39,  42,  44,  45,
         47,  49,  50,  52,  55,  57,  58,  60,  63,  64,  65,  66,  68,
         69,  70,  72,  79,  81,  82,  87,  89,  90,  91,  93,  95,  96,
        100, 101, 102, 103, 105, 106, 107, 108, 109, 110, 111, 112, 113,
        115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
        128, 130, 131]),
 array([ 5,  3,  2,  2,  7,  1,  1,  1,  2,  1,  1,  1,  2,  1,  1,  2,  1,
         1,  1,  3,  1,  1,  5,  1,  4,  1,  1,  1,  1,  1,  1,  3,  2,  1,
         1,  4,  7,  3,  2,  2,  2,  3,  1,  1,  1,  2,  1,  2,  2,  2,  1,
         1,  1,  1,  2,  7, 31,  5,  2,  3,  8,  4,  4, 15,  5,  4,  3,  8,
         6,  7, 12,  4,  4,  1,  9,  6,  1,  1,  3,  1,  1]))

(81,)

In [144]:
%%time
multi_model = LGBMClassifier(learning_rate=0.2)

multi_model.fit(x_train, y_train)
y_pred = multi_model.predict(x_val)

CPU times: user 46.5 s, sys: 238 ms, total: 46.7 s
Wall time: 28.9 s


In [145]:
precision_score(y_val, y_pred, average='macro')

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.05727457339151492

### (3) 챗봇 구축
* 아래 절차대로 수행하는 함수 만들기
    * input 질문 
    * 1단계 : 모델을 이용하여 type 0, 1로 분류
    * 2단계 : 
        * train의 모든 Q와 input 문장의 임베딩 벡터간의 코사인 유사도 계산
        * 코사인 유사도가 가장 높은 Q를 선택
        * 선택한 Q의 intent에 맵핑된 답변 중 하나를 무작위로 선택

#### 1) 하나의 질문으로 테스트해보기

* 선택된 질문과 답변

In [114]:
import random

question = input("type your question here: ")

# 토큰화된 데이터로 word2vec 학습
question = [tokenize(get_tokenizer('mecab'), question)]

# 학습된 word2vec 모델로 데이터셋 구성
inputquestion = get_dataset(question, pre_wv_model, 200)

predicted_intent = model.predict(inputquestion)

# 예측된 intent에 해당하는 답변들의 리스트 가져오기
response_options = train[train['intent'] == predicted_intent[0]]['A'].tolist()

# 답변들 중에서 랜덤하게 하나 선택
response = random.choice(response_options)

print('답변: ', response)
# print(response_options)

type your question here: 예비군 훈련 참석시 불참으로 처리되나요?
답변:  단위기간(훈련시작일로부터 1개월) 내 80% 이상 출석해야 훈련수강 유지가 가능합니다. 
예를 들어, 훈련시작일로부터 1개월을 기준으로 1개월간의 훈련일수가 20일이라고 가정할 경우 16일 이상 출석하셔야 훈련수강을 계속 유지할 수 있습니다. 
결석 기준은 1. 당일 소정훈련시간의 50퍼센트 미만을 수강한 경우, 2. 지각, 조퇴 3회 누적 시, 결석 처리 됩니다. 
훈련 수준 유지를 위해 100% 출석을 권고 드립니다. 

K-Digital Training (K-DT) 규정상 월 1회 휴가 사용이 가능합니다.


* 예측을 위한 입력 형태로 변환
    * 학습을 위한 전처리 과정을 test 데이터에도 적용합니다. 

* 1단계 : type 분류

* 2단계 : 질문에 대한 벡터 만들고 코사인 유사도 계산
    * Word2Vec 사전 학습 모델로 부터 벡터 만들기

* train의 질문 벡터들과 유사도 계산
    * Word2Vec 으로 만든 벡터들과 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


#### 2) 챗봇 함수 만들기
* 위 테스트 결과를 바탕으로 코드를 정리하고 함수로 생성합니다.

In [ ]:
def get_answer2(question): 





    return

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

## 4.챗봇3

* **세부요구사항**
    * 단계별 챗봇을 만들어 봅시다.
        * 1단계 : 챗봇2의 1단계 모델을 사용합니다.
        * 2단계 : 
            * 각 type에 맞게, 사전학습된 Word2Vec 모델을 사용하여 임베딩 벡터(train)를 만들고
        * 3단계 : 챗봇 만들기
            * input 문장과 train 임베딩 벡터와 코사인 유사도 계산
            * 가장 유사도가 높은 질문의 intent 찾아
            * 해당 intent의 답변 중 무작위로 하나를 선정하여 답변하기

### (1) 1단계 : type 분류 모델링
- LSTM : 3-(1) 모델을 그대로 사용합니다.

### (2) FastText 모델

-  FastText 모델 학습을 위한 입력 포맷 2차원 리스트 형태 입니다.
  ```
  [['나', '는', '학생', '이다'], ['오늘', '은', '날씨', '가', '좋다']]
  ```

- Word2Vec계열의 FastText를 학습하는 이유
  - n-gram이 추가된 fasttext 모델은 유사한 단어에 대한 임베딩을 word2vec보다 잘 해결할 수 있으며, 오탈자 등에 대한 임베딩 처리가 가능하다.
  - 예) 체크카드, 쳌카드는 word2vec에서는 전혀 다른 단어이지만 fasttext는 character n-gram으로 비교적 같은 단어로 처리할 수 있다.
- 참고: https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText


#### 1) 데이터 준비
* 시작데이터 : clean_train_questions, clean_test_questions

* FastText를 위한 입력 데이터 구조 만들기

#### 2) FastText 모델 생성
* FastText 문법
    * FastText( input데이터,  min_count = , size= , window=  )
        * input데이터 : 학습에 사용할 문장으로 이루어진 리스트
        * min_count : 모델에 사용할 단어의 최소 빈도수. 이 값보다 적게 출현한 단어는 모델에 포함되지 않음. 기본값 = 5
        * size : 단어의 벡터 차원 지정. 기본값 = 100
        * window : 학습할 때 한 단어의 좌우 몇 개의 단어를 보고 예측을 할 것인지를 지정. 기본값 = 5
    * 참조 : https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText

In [ ]:
from gensim.models.fasttext import FastText
import gensim.models.word2vec



#### 3) train에 임베딩벡터 결과 저장
* get_sent_embedding 함수를 이용하여 train의 질문별 임베딩 결과를 저장합니다.
    * .apply(lambda .....) 를 활용하세요.

### (3) 챗봇 구축
- input 질문
- intent classifier로 common와 faq 중 하나를 예측
- 예측된 intent에 속한 train의 모든 Q와 input 문장의 임베딩 벡터간의 코사인 유사도 계산
- 코사인 유사도가 가장 높은 top-3개의 Q를 선택
- 선택한 Q에 맵핑된 답변 중 하나를 선택하고 실제 답변과 비교

#### 1) 하나의 질문으로 테스트해보기

* 선택된 질문과 답변

* 예측을 위한 입력 형태로 변환

* 예측하기

* 질문에 대한 벡터 만들기
    * FestText 모델로 부터 벡터 만들기

* train의 질문 벡터들과 유사도 계산
    * FastText 로 만들 벡터들과 유사도 계산

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity






#### 2) 함수로 생성하기

In [ ]:
def get_answer3(question): 







    return 

#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

## 5.질문에 대한 답변 비교해보기

* **세부요구사항**
    * 세가지 챗봇을 생성해 보았습니다. 
    * 질문을 입력하여 답변을 비교해 봅시다. 어떤 챗봇이 좀 더 정확한 답변을 하나요?
